In [ ]:
from arcgis.gis import GIS
import pandas as pd
# from arcgis.features import FeatureSet

from slugify import slugify
from pathlib import Path

In [ ]:
gis = GIS("https://geoportal.sgb.gov.br/portal/")
gis

# FeatureLayerCollection

In [ ]:
fc = gis.content.get('dc0ec5801eff48ec98924a609f939cbc')
fc

In [ ]:
[(index, layer.properties.name) for index, layer in enumerate(fc.layers) if layer.properties.relationships]

## Começar daqui

In [ ]:
survey_fc = fc.layers[8]
survey_fc.properties.name

In [ ]:
%%time
survey_df = survey_fc.query(
    # "projeto_publicacao = 'Folha Petrolina'",
    return_geometry=False
).sdf.set_index("objectid")

survey_df.info()

In [ ]:
# Identificar tabelas relacionadas
related_tables = {}  # index: table

for relationship in survey_fc.properties.relationships:
    assay_tbl = [(k, t.properties.name) for k, t in enumerate(fc.tables) if t.properties.id == relationship.relatedTableId]
    table_id, table_name = assay_tbl[0]
    related_tables[table_id] = table_name

related_tables

In [ ]:
result_df = survey_df.join(
    pd.concat([
        fc.tables[i]
            .query()
            .sdf  
            .filter(
                ["amostra", "analito", "valor"]
            )
            .set_index(
                ["amostra", "analito"]
            )
            .squeeze()
            .unstack()
            .rename(columns=lambda col: slugify(col, separator="_"))
            .sort_index(axis="columns")
            .replace({
                "S1" : "<1%",
                "S3" : "1-5%",
                "S15": "5-25%",
                "S40": "25-50%",
                "S60": "50-75%",
                "S85": ">75%",
                "X"  : ">50%",
                "Y"  : "5-50%",
                "Z"  : "<5%",
            })
            .rename_axis("objectid")
        for i in related_tables.keys()
    ], axis="columns")        
)

out_csv = Path("output").joinpath(slugify(table_name, separator="_") + ".csv")
result_df.to_csv(out_csv, index=True)

out_csv

In [ ]:
result_df